In [52]:

#UTILITIES
import pandas as pd 
import numpy as np 
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from ast import literal_eval
import itertools
import time
import re

#MACHINE LEARNING
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, VotingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

#MAKE CHARTS PRETTY
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Arial'],'size':'14'}) 

#Some Settings

pd.set_option('display.max_columns', 100)

RANDOM_STATE = len('I used to be an adventurer like you') #But I took an arrow to the knee.

In [94]:
#Data import

#Kaggle Dataset of 200k steam games by user (this will be used for recommendations)
steam_200k = pd.read_csv('data/Kaggle/steam-200k.csv')
#Dataset gathered using steam API
raw_steam = pd.read_csv('steam_app_data.csv')
#Steam Spy API
steam_spy = pd.read_csv('data/Kaggle/steamspy_data.csv')
#Proprietary Dataset
steam_df = pd.read_csv('data/Kaggle/df_for_data.csv')

In [95]:
raw_steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37726 entries, 0 to 37725
Data columns (total 39 columns):
type                       37630 non-null object
name                       37723 non-null object
steam_appid                37726 non-null int64
required_age               37630 non-null float64
is_free                    37630 non-null object
controller_support         7856 non-null object
dlc                        6470 non-null object
detailed_description       37609 non-null object
about_the_game             37608 non-null object
short_description          37609 non-null object
fullgame                   0 non-null float64
supported_languages        37607 non-null object
header_image               37630 non-null object
website                    23648 non-null object
pc_requirements            37630 non-null object
mac_requirements           37630 non-null object
linux_requirements         37630 non-null object
legal_notice               12746 non-null object
drm_notice    

In [96]:
raw_steam.head()

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Counter-Strike,10,0.0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,NaN,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 999, 'final': 9...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 92459},NaN,"{'coming_soon': False, 'date': 'Nov 1, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 499, 'final': 4...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 3491},NaN,"{'coming_soon': False, 'date': 'Apr 1, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,game,Day of Defeat,30,0.0,False,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,NaN,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 499, 'final': 4...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 2634},NaN,"{'coming_soon': False, 'date': 'May 1, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}"
3,game,Deathmatch Classic,40,0.0,False,NaN,NaN,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: 

In [97]:
metacritic_raw = raw_steam[['name', 'steam_appid', 'metacritic', 'reviews', 'recommendations']]

## Data Cleaning

In [98]:
null = raw_steam.isnull().sum()
null

type                          96
name                           3
steam_appid                    0
required_age                  96
is_free                       96
controller_support         29870
dlc                        31256
detailed_description         117
about_the_game               118
short_description            117
fullgame                   37726
supported_languages          119
header_image                  96
website                    14078
pc_requirements               96
mac_requirements              96
linux_requirements            96
legal_notice               24980
drm_notice                 37522
ext_user_account_notice    37000
developers                   201
publishers                    96
demos                      34927
price_overview              5702
packages                    5173
package_groups                96
platforms                     96
metacritic                 34374
reviews                    30731
categories                   854
genres    

There are severl columns missing more than half of the information. Since these wont help us we're going to drop each of these columns. 

In [99]:
#Set a threshold for dropping null values

threshold = raw_steam.shape[0] // 2
print('the threshold is:', threshold)

the threshold is: 18863


In [100]:
#Get a list of columns to drop
drop_col = raw_steam.columns[null > threshold]

In [101]:
print('Columns to drop: {}'.format(list(drop_col)))

Columns to drop: ['controller_support', 'dlc', 'fullgame', 'legal_notice', 'drm_notice', 'ext_user_account_notice', 'demos', 'metacritic', 'reviews', 'recommendations']


In [102]:
print('Rows to remove:', raw_steam[raw_steam['type'].isnull()].shape[0])

raw_steam[raw_steam['type'].isnull()].head()

Rows to remove: 96


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
36,NaN,ValveTestApp852,852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,NaN,Star Trek: DAC - Demo,4330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266,NaN,Puzzlegeddon,8740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274,NaN,Borderlands DLC: Claptrap's New Robot Revolution,8955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
397,NaN,America's Army 3 Beta,13120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
#Check Value Counts
raw_steam['type'].value_counts(dropna=False)

game    37630
NaN        96
Name: type, dtype: int64

This matches the above so we're cleared to proceed

In [104]:
#Moving on to name column, check for missing game names

missing_names = raw_steam[(raw_steam['name'].isnull()) | (raw_steam['name'] == 'none')]
missing_names

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
489,NaN,NaN,17760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4928,game,none,339860,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,English<strong>*</strong><br><strong>*</strong...,https://steamcdn-a.akamaihd.net/steam/apps/339...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],NaN,NaN,NaN,NaN,[''],NaN,NaN,NaN,[],"{'windows': True, 'mac': False, 'linux': False}",NaN,NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '25', 'description': 'Adventure'}, {'i...",NaN,NaN,NaN,"{'total': 3, 'highlighted': [{'name': 'Master ...","{'coming_soon': False, 'date': 'Feb 27, 2015'}","{'url': '', 'email': ''}",NaN,"{'ids': [], 'notes': None}"
6789,game,none,385020,0.0,False,NaN,NaN,- discontinued - (please remove),- discontinued - (please remove),- discontinued - (please remove),NaN,"English, French, Italian, German, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/385...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,NaN,NaN,NaN,['none'],[''],NaN,NaN,NaN,[],"{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...",NaN,NaN,NaN,NaN,"{'coming_soon': False, 'date': 'Nov 4, 2015'}","{'url': '', 'email': ''}",NaN,"{'ids': [], 'notes': None}"
7239,game,NaN,396420,0.0,True,NaN,NaN,Spookeningは3Dの恐怖ゲームで、あなたは毎夜に死んでゴーストとして復活します。<b...,Spookeningは3Dの恐怖ゲームで、あなたは毎夜に死んでゴーストとして復活します。<b...,Spookeningは3Dの恐怖ゲームで、あなたは毎夜に死んでゴーストとして復活します。 村...,NaN,NaN,https://steamcdn-a.akamaihd.net/steam/apps/396...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],NaN,NaN,NaN,NaN,[''],NaN,NaN,NaN,[],"{'windows': True, 'mac': False, 'linux': False}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coming_soon': False, 'date': 'Nov 1, 2016'}","{'url': '', 'email': ''}",NaN,"{'ids': [], 'notes': None}"
7352,game,none,398970,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,English<strong>*</strong><br><strong>*</strong...,https://steamcdn-a.akamaihd.net/steam/apps/398...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,NaN,NaN,NaN,['none'],['none'],"[{'appid': 516340, 'description': ''}]",NaN,NaN,[],"{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '25', 'description': 'Adventure'}, {'i...",NaN,NaN,NaN,"{'total': 35, 'highlighted': [{'name': ""They'v...","{'coming_soon': False, 'date': 'Nov 5, 2015'}","{'url': '', 'email': ''}",NaN,"{'ids': [], 'notes': None}"
34988,game,NaN,1216780,0.0,False,NaN,NaN,&quot;Our Journeys 2 ~ A Collection of Visual ...,&quot;Our Journeys 2 ~ A Collection of Visual ...,&quot;Our Journeys 2 ~ A Collection of Visual ...,NaN,"English, French",https://steamcdn-a.akamaihd.net/steam/apps/121...,http://miaqc.ca/en,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],NaN,NaN,NaN,NaN,[''],NaN,NaN,NaN,[],"{'windows': True, 'mac': False, 'linux': False}",NaN,NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256775407, 'name': 'Ixelda Shirina Tra...",NaN,"{'total': 20, 'highlighted': [{'name': 'Good E...","{'coming_soon': False, 'date': 'Mar 3, 2020'}","{'url': 'http://miaqc.ca/en', 'emai

Only six observations missing names (and a lot of other info) so we'll drop these too

In [105]:
duplicate_rows = raw_steam[raw_steam.duplicated()]

print('Dupicate rows to remove:', duplicate_rows.shape[0])

duplicate_rows.head()

Dupicate rows to remove: 16


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
10,game,Counter-Strike,10,0.0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,NaN,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 999, 'final': 9...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 92459},NaN,"{'coming_soon': False, 'date': 'Nov 1, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
11,game,Team Fortress Classic,20,0.0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 499, 'final': 4...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 3491},NaN,"{'coming_soon': False, 'date': 'Apr 1, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
12,game,Day of Defeat,30,0.0,False,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,NaN,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 499, 'final': 4...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 2634},NaN,"{'coming_soon': False, 'date': 'May 1, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}"
13,game,Deathmatch Classic,40,0.0,False,NaN,NaN,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minim

Now its time to treat the null values (be gone demons)

In [106]:
def drop_null_cols(df, thresh=0.5):
    '''Drop columns with null values over the established threshold'''
    cutoff = len(df) * thresh
    
    return df.dropna(thresh=cutoff, axis = 1)

def drop_name_type(df):
    '''Drop rows with null values and none for game type and name'''
    #remove rows where type col is blank
    df = df[df['type'].notnull()]
    #remove rows where name is not defined
    df = df[df['name'].notnull()]
    df = df[df['name'] != 'none']
    #remove type col
    df = df.drop('type', axis=1)
    
    return df

def process(df):
    '''Process the dataframe to clean and use functions we create'''
    
    #copy df to avoid unhappy accidents
    df = df.copy()
    
    #Remove duplicate rows
    df = df.drop_duplicates()
    
    #Remove col with more than 50% null val
    df = drop_null_cols(df)
    
    #Remove rows with with null vals for game and type then drop type col
    df = drop_name_type(df)
    
    return df



In [107]:
print(raw_steam.shape)
initial_df = process(raw_steam)
print(initial_df.shape)
initial_df.head()

(37726, 39)
(37609, 28)


,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
0,Counter-Strike,10,0.0,False,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'USD', 'initial': 999, 'final': 9...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Nov 1, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,Team Fortress Classic,20,0.0,False,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'USD', 'initial': 499, 'final': 4...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Apr 1, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,Day of Defeat,30,0.0,False,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'USD', 'initial': 499, 'final': 4...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'May 1, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}"
3,Deathmatch Classic,40,0.0,False,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'USD', 'initial': 499, 'final': 4...",[31],"[{'name': 'default', 'title': 'Buy Deathmatch ...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{

# Age

The next column to look at is required_age. 

In [108]:
null_df = initial_df.isnull().sum()
print('empty age requirements:', null_df['required_age'])

empty age requirements: 0


In [109]:
initial_df['required_age'].value_counts(dropna=False).sort_index()

0.0     36584
1.0         1
3.0         3
5.0         1
6.0         4
7.0         3
10.0       22
11.0        1
12.0       25
13.0      146
14.0        3
15.0        6
16.0       52
17.0      583
18.0      172
20.0        1
21.0        2
Name: required_age, dtype: int64

The range of age is between 0 and 21.

Video games are a largely self regulated industry. In the US, ESRB puts ratings on games for recommended ages. In the rest of the world, PEGI puts ratings on games. ESRB uses letters (E, E10+, T, M) and PEGI uses numbers for ages (3, 7, 12, 16, 18). 

Binning by ages seeems more appropriate here so we'll use PEGI system.

https://www.esrb.org/

https://pegi.info/


In [110]:
initial_df[initial_df['required_age'] == 21]

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
22758,Booty Calls,823550,21.0,True,Help mermaid princess Andriella fulfill her ta...,Help mermaid princess Andriella fulfill her ta...,Help mermaid princess Andriella fulfill her ta...,"English<strong>*</strong>, French, German, Spa...",https://steamcdn-a.akamaihd.net/steam/apps/823...,https://www.bootycallsgame.com/,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],['3X Entertainment Limited'],['3X Entertainment Limited'],NaN,NaN,[],"{'windows': True, 'mac': True, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256723441, 'name': 'Teaser', 'thumbnai...",NaN,"{'coming_soon': False, 'date': 'Sep 18, 2018'}","{'url': 'http://support.bootycallsgame.com', '...",https://steamcdn-a.akamaihd.net/steam/apps/823...,"{'ids': [1, 3, 5], 'notes': 'The content of Bo..."
35579,Hades Ultimate Fighting Ball,1240680,21.0,False,Hades Ultimate Fighting Ball is a Multiplayer ...,Hades Ultimate Fighting Ball is a Multiplayer ...,Hades Ultimate Fighting Ball is a multiplayer ...,"French<strong>*</strong>, English, Spanish - S...",https://steamcdn-a.akamaihd.net/steam/apps/124...,https://hades-game.com/,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,"['Mustieles Irvin', 'Atenza Nicolas']","['Mustieles Irvin', 'Atenza Nicolas']","{'currency': 'USD', 'initial': 599, 'final': 5...",[430581],"[{'name': 'default', 'title': 'Buy Hades Ultim...","{'windows': True, 'mac': False, 'linux': False}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256778899, 'name': 'Hades UFB shortvid...",NaN,"{'coming_soon': False, 'date': 'Mar 31, 2020'}","{'url': 'https://hades-game.com/', 'email': 's...",https://steamcdn-a.akamaihd.net/steam/apps/124...,"{'ids': [1, 2, 5], 'notes': 'Violence, coarse ..."


In [111]:
def pegi_age(df):
    '''Bin age requirments according to PEGI standards'''
    #PEGI Age Ratings 3, 7, 12, 16, 18
    
    bins = [-1, 0, 3, 7, 12, 16, 2000]
    labels = [0, 3, 7, 12, 16, 18]
    
    df['required_age'] = pd.cut(df['required_age'], bins = bins, labels =labels)
    
    return df

In [112]:
age_df = pegi_age(initial_df)


In [113]:
age_df['required_age'].value_counts().sort_index()

0     36584
3         4
7         8
12       48
16      207
18      758
Name: required_age, dtype: int64

## Price (to be cont...)

The next column is whether the game is free to play or not displayed in boolean. It makes sense to deal with this when we deal with the price overview in the future. But lets take a sneak peak.

In [114]:
age_df['is_free'].value_counts()

False    33344
True      4265
Name: is_free, dtype: int64

In [115]:
age_df['price_overview'].head()

0    {'currency': 'USD', 'initial': 999, 'final': 9...
1    {'currency': 'USD', 'initial': 499, 'final': 4...
2    {'currency': 'USD', 'initial': 499, 'final': 4...
3    {'currency': 'USD', 'initial': 499, 'final': 4...
4    {'currency': 'USD', 'initial': 499, 'final': 4...
Name: price_overview, dtype: object

is_free column is next but it makes sense to treat this with the cost column so we'll skip ahead to platforms

## Platforms Column

Platforms on steam is what kind of Operating System the game can be played (Windows, MAC OS, linux)

In [116]:
age_df['platforms'].head()

0    {'windows': True, 'mac': True, 'linux': True}
1    {'windows': True, 'mac': True, 'linux': True}
2    {'windows': True, 'mac': True, 'linux': True}
3    {'windows': True, 'mac': True, 'linux': True}
4    {'windows': True, 'mac': True, 'linux': True}
Name: platforms, dtype: object

This column has a dictionary structure with keys representing the operating system and values representing whether the game can be played on that os. 

In [117]:
null_df['platforms']

0

No null values to worry about in this column.

In [118]:
def clean_platforms(df):
    '''Split platforms column into seperate boolean columns for each platform'''
    #evaluate values in platforms column
    df = df.copy()
    
    def parse_platforms(x):
        
        d = literal_eval(x)
        
        return ';'.join(platform for platform in d.keys() if d[platform])
    
    df['platforms'] = df['platforms'].apply(parse_platforms)
    
    return df

In [119]:
platforms_df = clean_platforms(age_df)
platforms_df['platforms'].value_counts()

windows              26423
windows;mac;linux     5533
windows;mac           4648
windows;linux          997
mac                      6
mac;linux                1
linux                    1
Name: platforms, dtype: int64

Majority of games are only availabe on windows. With only one game available exclusively on linux. 

In [120]:
platforms_df.head()

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
0,Counter-Strike,10,0,False,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'USD', 'initial': 999, 'final': 9...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...",windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Nov 1, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,Team Fortress Classic,20,0,False,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'USD', 'initial': 499, 'final': 4...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...",windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Apr 1, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,Day of Defeat,30,0,False,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'USD', 'initial': 499, 'final': 4...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...",windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'May 1, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}"
3,Deathmatch Classic,40,0,False,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'USD', 'initial': 499, 'final': 4...",[31],"[{'name': 'default', 'title': 'Buy Deathmatch ...",windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Jun 1, 2001'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/40/...,"{'id

## Price (For real this time)

In [121]:
null_df['price_overview']

5599

Before writing this off as a loss, lets see what gaps we can fill with the is_free column

In [122]:
free_null_price_index = platforms_df[(platforms_df['is_free']) & (platforms_df['price_overview'].isnull())]

free_null_price_index.shape[0]

4211

In [123]:
not_free_null_price = platforms_df[(platforms_df['is_free'] == False) & (platforms_df['price_overview'].isnull())]

not_free_null_price

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
73,The Ship: Single Player,2420,0,False,For PC gamers who enjoy multiplayer games with...,For PC gamers who enjoy multiplayer games with...,The Ship is a murder mystery alternative to tr...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/242...,http://www.blazinggriffin.com/games/the-ship-m...,{'minimum': '<strong>Minimum:</strong> 1.8 GHz...,[],[],['Outerlight Ltd.'],['Blazing Griffin Ltd.'],NaN,[56669],"[{'name': 'default', 'title': 'Buy The Ship: S...",windows,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 2035597, 'name': 'the Ship: Intro', '...",NaN,"{'coming_soon': False, 'date': 'Nov 20, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/242...,"{'ids': [], 'notes': None}"
85,RollerCoaster Tycoon® 3: Platinum,2700,0,False,Rollercoaster Tycoon 3 Platinum combines the e...,Rollercoaster Tycoon 3 Platinum combines the e...,Rollercoaster Tycoon 3 Platinum combines the e...,"English, French, Italian, German, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/270...,http://www.atari.com/rollercoastertycoon/us/in...,{'minimum': '<strong>Minimum: </strong><br>\t\...,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],"['Frontier', 'Aspyr (Mac)']","['Atari', 'Aspyr (Mac)']",NaN,NaN,[],windows;mac,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '28', 'description': 'Simulation'}, {'...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Mar 12, 2008'}","{'url': 'http://www.atari.com/support/atari', ...",https://steamcdn-a.akamaihd.net/steam/apps/270...,"{'ids': [], 'notes': None}"
116,Hammer Heads Deluxe,3400,0,False,Pesky yard gnomes are invading your lawn! The ...,Pesky yard gnomes are invading your lawn! The ...,Pesky yard gnomes are invading your lawn! The ...,English,https://steamcdn-a.akamaihd.net/steam/apps/340...,NaN,{'minimum': '<strong>Minimum Requirements:</st...,[],[],"['PopCap Games, Inc.']","['PopCap Games, Inc.']",NaN,NaN,[],windows,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '4', 'description': 'Casual'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Aug 30, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/340...,"{'ids': [], 'notes': None}"
120,Rocket Mania Deluxe,3440,0,False,Can you become master of pyrotechnics? Twist ...,Can you become master of pyrotechnics? Twist ...,Can you become master of pyrotechnics? Twist a...,English,https://steamcdn-a.akamaihd.net/steam/apps/344...,NaN,{'minimum': '<strong>Minimum Requirements:</st...,[],[],"['PopCap Games, Inc.']","['PopCap Games, Inc.']",NaN,NaN,[],windows,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '4', 'description': 'Casual'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Aug 30, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/344...,"{'ids': [], 'notes': None}"
230,BioShock™,7670,18,False,<h1>Special Offer</h1><p>Buying BioShock™ also...,BioShock is a shooter unlike any you've ever p...,BioShock is a shooter unlike any you've ever p...,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/767...,http://www.BioShockGame.com,"{'minimum': '<h2 class=""bb_tag""><strong>Minimu...",{'minimum': 'Please See BioShock Remastered'},[],"['2K Boston', '2K Australia']",['2K'],NaN,"[451, 127633]","[{'name': 'default', 'title': 'Buy BioShock™',...",windows,"[{'

846 games in our dataframe do not have a price listed. This is odd so lets investigate a handful.

In [124]:
not_free_sample = not_free_null_price.sample(n=5, random_state=RANDOM_STATE)

https://store.steampowered.com/app

In [125]:
def print_steam_links(df):
    url= 'https://store.steampowered.com/app/'
    
    for i, row in df.iterrows():
        appid = row['steam_appid']
        name = row['name']
        
        print(name + ':', url + str(appid))
    

In [126]:
print_steam_links(not_free_sample)

Marvel: Ultimate Alliance 2: https://store.steampowered.com/app/433320
R.B.I. Baseball 16: https://store.steampowered.com/app/407690
IL-2 Sturmovik: Cliffs of Dover: https://store.steampowered.com/app/63950
VR Golf Online: https://store.steampowered.com/app/575540
Grandpa and the Zombies: https://store.steampowered.com/app/732390


* Eagle Island is a demo page

* Rain of Reflections was in development at time fo data collection

* Holobunnies does not have a price listed as it is in active development

* Norman's Great Illusion was in development at time of data collection 

* Viscera Cleanup Detail: Shadow Warrior Multiple editions available

For various reasons, the games do not have a price. We'll push off treating these for now as there might be other reasons they get dropped. 


In [131]:
platforms_df['price_overview'][5]

"{'currency': 'USD', 'initial': 499, 'final': 499, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': '$4.99'}"

In [132]:
def clean_price(df):
    df = df.copy()
    
    def parse_price(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return {'currency': 'USD', 'initial': -1}
    #evaluate as dictionary and set to -1 if missing
    df['price_overview'] = df['price_overview'].apply(parse_price)
    
    # Create columns from currency and initial values
    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price']    = df['price_overview'].apply(lambda x: x['initial'])
    
    # Free games are free
    df.loc[df['is_free'], 'price'] = 0 
    
    return df
        

In [133]:
price_data = clean_price(platforms_df)[['name', 'currency', 'price']]

price_data.head()

,name,currency,price
0,Counter-Strike,USD,999
1,Team Fortress Classic,USD,499
2,Day of Defeat,USD,499
3,Deathmatch Classic,USD,499
4,Half-Life: Opposing Force,USD,499


In [134]:
price_data[price_data['currency'] != 'USD']

,name,currency,price
19286,Roguebreaker,RUB,3000


There is one game that is not listed in USD, since the number is so small we'll markfordelete

In [137]:
def clean_price(df):
    '''clean price_overview col into formatted price column'''
    df = df.copy()
    
    def parse_price(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return{'currency': 'USD', 'initial': -1}
    # eval as dictionary and set to -1 if missing
    df['price_overview'] = df['price_overview'].apply(parse_price)
    
    # create columns from currency and initial values
    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price'] = df['price_overview'].apply(lambda x: x['initial'])
    
    # set price of free games to 0
    df.loc[df['is_free'], 'price'] = 0 
    
    #remove non-USD rows
    df = df[df['currency']== 'USD']
    
    # Change price to display in pounds
    df.loc[df['price'] > 0, 'price']/= 100
    
    #remove columns no longer needed
    df = df.drop(['is_free', 'currency', 'price_overview'], axis=1)
    
    return df

In [138]:
price_df = clean_price(platforms_df)
price_df[['name', 'price']].head()

,name,price
0,Counter-Strike,9.99
1,Team Fortress Classic,4.99
2,Day of Defeat,4.99
3,Deathmatch Classic,4.99
4,Half-Life: Opposing Force,4.99


## Packages

In [139]:
with pd.option_context('display.max_colwidth', 500):
    display(price_df[['steam_appid', 'packages', 'package_groups', 'price']].head())

,steam_appid,packages,package_groups,price
0,10,[7],"[{'name': 'default', 'title': 'Buy Counter-Strike', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 7, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Counter-Strike: Condition Zero - $9.99', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 999}]}]",9.99
1,20,[29],"[{'name': 'default', 'title': 'Buy Team Fortress Classic', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 29, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Team Fortress Classic - $4.99', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 499}]}]",4.99
2,30,[30],"[{'name': 'default', 'title': 'Buy Day of Defeat', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 30, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Day of Defeat - $4.99', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 499}]}]",4.99
3,40,[31],"[{'name': 'default', 'title': 'Buy Deathmatch Classic', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 31, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Deathmatch Classic - $4.99', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 499}]}]",4.99
4,50,[32],"[{'name': 'default', 'title': 'Buy Half-Life: Opposing Force', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 32, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Opposing Force - $4.99', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 499}]}]",4.99


In [140]:
print(price_df[price_df['price']== -1].shape[0])

1388


In [141]:
print('Null counts:', price_df['package_groups'].isnull().sum())
print('Empty list counts:', price_df[price_df['package_groups']=='[]'].shape[0])

Null counts: 0
Empty list counts: 5341


In [142]:
missing_price_package = price_df[(price_df['price'] == -1) & (price_df['package_groups'] == '[]')]

print('Number of rows:', missing_price_package.shape[0], '\n')

print('First few rows:\n')
print_steam_links(missing_price_package[:5])

print('\nLast few rows:\n')
print_steam_links(missing_price_package[-10:-5])

Number of rows: 1342 

First few rows:

RollerCoaster Tycoon® 3: Platinum: https://store.steampowered.com/app/2700
Hammer Heads Deluxe: https://store.steampowered.com/app/3400
Rocket Mania Deluxe: https://store.steampowered.com/app/3440
Telltale Texas Hold ‘Em: https://store.steampowered.com/app/8330
Strong Bad's Cool Game for Attractive People: Season 1: https://store.steampowered.com/app/8340

Last few rows:

Tabletop Playground Beta: https://store.steampowered.com/app/1282050
Eternal Vortex - Beta: https://store.steampowered.com/app/1287980
Prophecy: https://store.steampowered.com/app/1289800
Project Océara: https://store.steampowered.com/app/1290820
梦中女孩: https://store.steampowered.com/app/1292790


In [143]:
def process_price(df):
    
    """Process price_overview column into formatted price column, and take care of package columns."""
    df = df.copy()
    def parse_price(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return {'currency': 'USD', 'initial': -1}
    # evaluate as dictionary and set to -1 if missing
    df['price_overview'] = df['price_overview'].apply(parse_price)
    # create columns from currency and initial values
    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price'] = df['price_overview'].apply(lambda x: x['initial'])
    # set price of free games to 0
    df.loc[df['is_free'], 'price'] = 0
    # remove non-USD rows
    df = df[df['currency'] == 'USD']
    # remove rows where price is -1
    df = df[df['price'] != -1]
    # change price to display in pounds (can apply to all now -1 rows removed)
    df['price'] /= 100
    # remove columns no longer needed
    df = df.drop(['is_free', 'currency', 'price_overview', 'packages', 'package_groups'], axis=1)
    
    return df



In [144]:
price_df = process_price(platforms_df)
price_df.head()

,name,steam_appid,required_age,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors,price
0,Counter-Strike,10,0,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Nov 1, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",9.99
1,Team Fortress Classic,20,0,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Apr 1, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",4.99
2,Day of Defeat,30,0,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'May 1, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}",4.99
3,Deathmatch Classic,40,0,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],windows;mac;linux,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,NaN,"{'coming_soon': False, 'date': 'Jun 1, 2001'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/40/...,"{'ids': [], 'notes': None}",4.99
4,Half-Life: Opposing Force,50,0,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,"English, French, German, Korean",https://steamcdn-a.akamaihd.net/steam/apps/50/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Gearbox Software'],['Valve'],win

## Languages

In [145]:
price_df['supported_languages'].isnull().sum()

7

In [146]:
price_df[price_df['supported_languages'].isnull()]

,name,steam_appid,required_age,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors,price
3990,Defense Grid 2: A Matter of Endurance,312860,0,NaN,NaN,NaN,NaN,https://steamcdn-a.akamaihd.net/steam/apps/312...,NaN,[],[],[],NaN,[''],windows,NaN,NaN,NaN,NaN,NaN,"{'coming_soon': False, 'date': 'Sep 22, 2014'}","{'url': '', 'email': ''}",NaN,"{'ids': [], 'notes': None}",5.99
4875,Subsiege,338640,0,"<img src=""https://steamcdn-a.akamaihd.net/stea...","<img src=""https://steamcdn-a.akamaihd.net/stea...",Subsiege is an intense real-time tactic game w...,NaN,https://steamcdn-a.akamaihd.net/steam/apps/338...,http://subsiege-game.com/,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],['Icebird Studios'],['Icebird Studios'],windows,NaN,NaN,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256729398, 'name': 'Release Trailer', ...",NaN,"{'coming_soon': False, 'date': 'Sep 7, 2018'}","{'url': 'http://subsiege-game.com/', 'email': ...",https://steamcdn-a.akamaihd.net/steam/apps/338...,"{'ids': [], 'notes': None}",19.99
14406,MARS VR(全球使命VR),596560,0,1.\t4K level audio-visual experience <br />\r\...,1.\t4K level audio-visual experience <br />\r\...,Welcome to 《Mars VR》. This is an immersive fir...,NaN,https://steamcdn-a.akamaihd.net/steam/apps/596...,http://qqsm.zygames.com/,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,"['Ying Pei Digital Technology Shanghai Co., Li...","['SHANGHAI ZHENYOU TECHNOLOGY CO.,LTD']",windows,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '73', 'description': 'Violent'}, {'id'...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256681371, 'name': 'marsvr', 'thumbnai...",NaN,"{'coming_soon': False, 'date': 'Apr 5, 2017'}","{'url': 'http://www.zygames.com/contact', 'ema...",https://steamcdn-a.akamaihd.net/steam/apps/596...,"{'ids': [], 'notes': None}",2.99
16243,Numberline 2,654970,0,NumberLine 2 is the continuation of the popula...,NumberLine 2 is the continuation of the popula...,NumberLine 2 is the continuation of the popula...,NaN,https://steamcdn-a.akamaihd.net/steam/apps/654...,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],"['V34D4R', 'Egor Magurin']",['Indovers Studio'],windows,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '4', 'description': 'Casual'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256687192, 'name': 'Numberline 2 Trail...","{'total': 60, 'highlighted': [{'name': '1st le...","{'coming_soon': False, 'date': 'Jul 14, 2017'}","{'url': '', 'email': 'radaew.zhenya@yandex.ru'}",https://steamcdn-a.akamaihd.net/steam/apps/654...,"{'ids': [], 'notes': None}",1.99
22006,It Could Have Been Me,803840,0,"<h2 class=""bb_tag"">It Could Have Been Me</h2><...","<h2 class=""bb_tag"">It Could Have Been Me</h2><...",A journey from Afganistan to Sweden. Experienc...,NaN,https://steamcdn-a.akamaihd.net/steam/apps/803...,https://www.facebook.com/1866040833678994/vide...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],"['Åsa Egnér vr-designer', 'Alexander Cobleigh ...",['Kreativitetsbanken'],windows,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '23', 'description': 'Indie'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...","[{'id': 256749583, 'name': 'it could have been...",NaN,"{'coming_soon': False, 'date': 'May 19, 2019'}","{'url': 'http://www.kreativitetsbanken.nu/', '...",https://steamcdn-a.akamaihd.net/steam/apps/803...,"{'ids': [], 'notes': None}",2.99
27010,SNUSE 221,948070,0,<strong> Hey. My name is *&amp;#!$.<br>Today I...,<strong> Hey. My name is *&amp;#!$.<br>Today I...,Hey. My name is *&amp;#!$. Today I will tell y...,NaN,https://steamcdn-a.akamaihd.net/steam/apps/948

In [147]:
print(price_df['supported_languages'][0])
price_df['supported_languages'].value_counts().head(10)

English<strong>*</strong>, French<strong>*</strong>, German<strong>*</strong>, Italian<strong>*</strong>, Spanish - Spain<strong>*</strong>, Simplified Chinese<strong>*</strong>, Traditional Chinese<strong>*</strong>, Korean<strong>*</strong><br><strong>*</strong>languages with full audio support


English                                                                                                        10499
English<strong>*</strong><br><strong>*</strong>languages with full audio support                                9377
English, Russian                                                                                                 768
English, Simplified Chinese                                                                                      418
English, Japanese                                                                                                365
Simplified Chinese                                                                                               308
Simplified Chinese<strong>*</strong><br><strong>*</strong>languages with full audio support                      243
English<strong>*</strong>, Russian<strong>*</strong><br><strong>*</strong>languages with full audio support      223
English, French, Italian, German, Spanish - Spain               

In [148]:
def process_language(df):
    """Process supported_languages column into a boolean 'is english' column."""
    df = df.copy()
    # drop rows with missing language data
    df = df.dropna(subset=['supported_languages'])
    df['english'] = df['supported_languages'].apply(lambda x: 1 if 'english' in x.lower() else 0)
    df = df.drop('supported_languages', axis=1)
    return df
language_df = process_language(price_df)
language_df[['name', 'english']].head()

,name,english
0,Counter-Strike,1
1,Team Fortress Classic,1
2,Day of Defeat,1
3,Deathmatch Classic,1
4,Half-Life: Opposing Force,1


In [149]:
language_df['english'].value_counts()

1    35249
0      964
Name: english, dtype: int64

## Developers and Publishers

In [150]:
print('Developers null counts:', language_df['developers'].isnull().sum())
print('Developers empty list counts:', language_df[language_df['developers'] == "['']"].shape[0])
print('\nPublishers null counts:', language_df['publishers'].isnull().sum())
print('Publishers empty list counts:', language_df[language_df['publishers'] == "['']"].shape[0])

Developers null counts: 67
Developers empty list counts: 0

Publishers null counts: 0
Publishers empty list counts: 236


In [151]:
no_dev = language_df[language_df['developers'].isnull()]

print('Total games missing developer:', no_dev.shape[0], '\n')

print_steam_links(no_dev.sample(n=5, random_state= RANDOM_STATE))

no_pub = language_df[language_df['publishers'] == "['']"]

print('\nTotal games missing publisher:', no_pub.shape[0], '\n')

print_steam_links(no_pub.sample(n=5, random_state= RANDOM_STATE))

no_dev_pub = language_df[(language_df['developers'].isnull()) & (language_df['publishers'] == "['']")]

print('\nTotal games missing developer and publisher:', no_dev_pub.shape[0], '\n')
print_steam_links(no_dev_pub.sample(n=5, random_state=RANDOM_STATE))

Total games missing developer: 67 

2 Planets Fire and Ice: https://store.steampowered.com/app/657110
Artemis Spaceship Bridge Simulator: https://store.steampowered.com/app/247350
Atlantic Quest Solitaire: https://store.steampowered.com/app/379260
Survival Tycoon: https://store.steampowered.com/app/667920
Path to the Sky: https://store.steampowered.com/app/370390

Total games missing publisher: 236 

RIP - Trilogy™: https://store.steampowered.com/app/2540
Forest Warrior: https://store.steampowered.com/app/378590
Crystal Cosmos: https://store.steampowered.com/app/496590
BattleStick: https://store.steampowered.com/app/394380
Tetradecagon: https://store.steampowered.com/app/487380

Total games missing developer and publisher: 27 

CRACKPOT DESPOT: TRUMP WARFARE: https://store.steampowered.com/app/910290
SAMOLIOTIK: https://store.steampowered.com/app/449680
Sign Here: _________: https://store.steampowered.com/app/845330
Fester Mudd: Curse of the Gold - Episode 1: https://store.steampowered

In [152]:
def clean_devs_pubs(df):
    '''clean developers and publisher columns'''
    
    #remove rows with missing data
    df = df[(df['developers'].notnull()) & (df['publishers'] != "['']")].copy()
    df = df[~(df['developers'].str.contains(';')) & ~(df['publishers'].str.contains(';'))]
    df = df[(df['publishers'] != "['NA']") & (df['publishers'] != "['N/A']")]
    
    #create list 
    df['developer'] = df['developers'].apply(lambda x: ';'.join(literal_eval(x)))
    df['publisher'] = df['publishers'].apply(lambda x: ';'.join(literal_eval(x)))
    
    df = df.drop(['developers', 'publishers'], axis = 1)
    
    return df

    

In [153]:
dev_pub_df = clean_devs_pubs(language_df)
dev_pub_df[['name', 'steam_appid', 'developer', 'publisher']].head()

,name,steam_appid,developer,publisher
0,Counter-Strike,10,Valve,Valve
1,Team Fortress Classic,20,Valve,Valve
2,Day of Defeat,30,Valve,Valve
3,Deathmatch Classic,40,Valve,Valve
4,Half-Life: Opposing Force,50,Gearbox Software,Valve


In [154]:
dev_pub_df.to_csv('checkpoint_df.csv')